In [9]:
import json, os, csv, time
import pandas as pd
import base64
import datetime
import urllib.request
import hashlib
import requests

In [2]:
with open("es_config.json", "r") as f:
    file_content = json.load(f)
username = file_content['user']
password = file_content['pass']

In [3]:
def send_json(data_json):
     ## Send json to ES with creds -- TODO: read ES creds from env file.
     url = "http://192.168.100.144:5400"
     body = json.dumps(data_json)
     jsondataasbytes = body.encode('utf-8')   # needs to be bytes

     req = urllib.request.Request(url, data=jsondataasbytes, method="POST")
     req.add_header('Content-Type', 'application/json; charset=utf-8')
     req.add_header('Content-Length', len(jsondataasbytes))
     creds = base64.b64encode(('f{username}:{password}').encode('utf-8'))
     req.add_header('Authorization', f'Basic {creds}')
     ## Add error handling and retry logic
     try:
          with urllib.request.urlopen(req) as response:
               response.close()
               if response.status != 200:
                    print(response.status, response.isclosed)
                    print(response.info())
                    while response.isclosed == False:
                         time.sleep(3)
                    time.sleep(1)
                    send_json(data_json)
     except urllib.error.HTTPError as err:
          print(err.code)
          return False, err
     except urllib.error.URLError as err:
          print(err.reason)
          return False, err
     except Exception as err:
          print(err)
          return False, err
     

     return True, None

In [34]:
def send_json(data_json, retry_count=0):
    ## Send json to ES with creds -- TODO: read ES creds from env file.
    url = "http://192.168.100.144:5400"
    body = json.dumps(data_json)
    try:
        res = requests.get(url, data=body, timeout=(10.0, 30))
        if res.status_code == 200:
            res.close()
            return True, None
        else:
            print("failed: " + str(res.status_code), res.text)
            time.sleep(3)
            return send_json(data_json, retry_count+1)
    except TimeoutError as e:
        print("timeout occured")
        print(e)
        time.sleep(10)
        return send_json(data_json, retry_count+1)
    except Exception as e:
        print("may be timeout occured")
        print(e)
        time.sleep(30)
        return send_json(data_json, retry_count+1)

In [35]:
headers = ['Time,Open,High,Low,Close']
m = hashlib.sha256()

with open("./USDJPY_forex_min1.csv", 'r') as f:
    reader = csv.reader(f)
    for i, ohlc in enumerate(reader):
        ## convert date str to date then add one day
        if i > 2144330:
            date_str = ohlc[0]
            date = datetime.datetime.fromisoformat(date_str)
            date_str = date.isoformat()+"Z"
            #delta = datetime.timedelta(days=1)
            
            ## Convert list to desired json format
            scheme = {
                "open": ohlc[1],
                "high": ohlc[2],
                "low": ohlc[3],
                "close": ohlc[4],
                "volume": 0,
                "provider": "forex",
                "type": "usd_jpy",
                "order": "mix",
                "scheme": "candle",
                "frame": "MIN1"
            }
            
            hash_org_str = f"USD_JPY forex candle MIN1 ${date_str}"
            hash_org = hash_org_str.encode()
            m.update(hash_org)
            hash_value = m.hexdigest()
            doc_id = hash_value
            scheme["doc_id"] = doc_id
            scheme["@timestamp"] = date_str

            suc, err = send_json(scheme)
            if suc:
                time.sleep(0.2)
            else:
                last_index = i
                print(f"error occured on {last_index}")
                break

In [33]:
i

2144333

['__abstractmethods__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_abc_impl', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_check_close', '_close_conn', '_get_chunk_left', '_method', '_peek_chunked', '_read1_chunked', '_read_and_discard_trailer', '_read_next_chunk_size', '_read_status', '_readall_chunked', '_readinto_chunked', '_safe_read', '_safe_readinto', 'begin', 'chunk_left', 'chunked', 'close', 'closed', 'code', 'debuglevel', 'detach', 'fileno', 'flush', 'fp', 'getcode', 'getheader', 'getheaders', 'geturl', 'headers', 'info', 'isatty', 'isclosed', 'length', 'msg', 'peek', 'read', 'read1', 'readable', 'readinto', 'readinto1', 'readline', 'readlines', 'reason', 'seek', 'seekable', 'status', 'tell', 'truncate', 'url', 'version', 'will_close', 'writable', 'write', 'writelines']